## Is a pre-trained GPT-2 model aware of island constraints?

Tokenizing sentences and calculating token-by-token surprisal uses helper functions of `minicons` package (https://github.com/kanishkamisra/minicons).

### Load required packages

In [ ]:
!pip install transformers==4.10.0 torch==1.8.1 sentencepiece==0.1.96 minicons

In [ ]:
from minicons import scorer
import torch
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
from transformers import pipeline,AutoTokenizer,TextDataset,DataCollatorForLanguageModeling,Trainer,TrainingArguments,AutoModelWithLMHead
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import json
from scipy import stats

In [ ]:
#access files saved in Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### GPT-2 pre-trained model
I am using a Japanese GPT-2 model trained by [Colorful Scoop](https://colorfulscoop.com/).

In [ ]:
pretrained_model = scorer.IncrementalLMScorer("colorfulscoop/gpt2-small-ja", 'cpu')

Downloading:   0%|          | 0.00/328 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/803k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/129 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/940 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/454M [00:00<?, ?B/s]

In [ ]:
def batch_surprisal_transformer(model, data):
  dataset = pd.DataFrame(columns = ["island", "sent_number", "token", "surprisal"])
  sent_number = 0
  for i, row in data.iterrows():
    tokens = []
    surprisals = []
    sent_number += 1
    results = model.token_score(row['sentence'], surprisal = True, base_two = True)
    for i in results:
      for j in i:
        tokens.append(j[0])
        surprisals.append(j[1])
    if row['island'] == 1:
      to_append = {'island': list(np.repeat(1, len(tokens), axis=0)),
                   'sent_number': list(np.repeat(sent_number, len(tokens), axis=0)),
                   'token': tokens,
                   'surprisal': surprisals}
      dataset = dataset.append(pd.DataFrame(to_append))
    elif row['island'] == 0:
      to_append = {'island': list(np.repeat(0, len(tokens), axis=0)),
                   'sent_number': list(np.repeat(sent_number, len(tokens), axis=0)),
                   'token': tokens,
                   'surprisal': surprisals}
      dataset = dataset.append(pd.DataFrame(to_append))
  return dataset

Import test data, and compute token-by-token surprisal using the LM

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/[file path]/test_stimuli.csv') # change path names before running

In [ ]:
#get and save the result
result = batch_surprisal_transformer(pretrained_model, data)
result.to_csv('result_GPT2.csv')

After exporting the result, I manually coded the region of interest (i.e., all the words following the noun that underwent long-distance relativization), and reimported the edited version.

In [ ]:
result_edited = pd.read_csv('/content/drive/MyDrive/[file path]/result_GPT2_pretrained_0131.csv') # change path names before running

In [ ]:
cond_list = []
for i, row in result_edited.iterrows():
  if row['sent_number'] < 9:
    cond_list.append('noext')
  elif row['sent_number'] > 16:
    cond_list.append('ext_isl')
  else:
    cond_list.append('ext_noisl')

result_edited['condition'] = cond_list

In [ ]:
#get mean surprisal on the critical region by group (island=1/non-island=0)
critical = result_edited[result_edited['critical']==1]
critical.groupby('condition')['surprisal'].mean()

condition
ext_isl      9.132382
ext_noisl    9.213679
noext        7.145632
Name: surprisal, dtype: float64

## Test for significance
I am using paired t-tests to see whether the difference in mean suprisal observed above is statistically significant.

In [ ]:
from scipy.stats import ttest_rel

alpha = 0.05

group_comparisons = [(noext, ext_noisl), (ext_noisl, ext_isl), (ext_isl, noext)]

for (groupA, groupB) in group_comparisons:
    stat, p = ttest_rel(groupA, groupB)
    corrected_p = p * len(group_comparisons)

    print(stat, p)

-4.316838347969772 7.880356676364425e-05
0.5185723158494386 0.6064415427495102
4.543388622769791 3.743011834239794e-05


The results suggest that GPT-2 trained entirely on Japanese dataset assigns higher surprisals to words that have undergone long-distance extraction out of an island, compared with words that have just undergone long-distance extraction (no island violation). In other words, it is possible that GPT-2 has learned the relative clause island constraint.